In [1]:
# Misc imports
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
import torch
torch.set_printoptions(linewidth=200)
import seaborn as sns
sns.set_style("darkgrid")
import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
import yaml
from pathlib import Path
# Local imports
from ese.scripts.utils import get_option_product
from ese.experiment.analysis.run_inference import get_cal_stats
from ese.experiment.analysis.ensemble_upperbound import get_ensemble_ub
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
scratch_root = Path("/storage/vbutoi/scratch/ESE")
code_root = Path("/storage/vbutoi/projects/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

/local/vbutoi/envs/UniverSegTF/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_outputs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
%%yaml inference_config

log:
    root: '?'
    min_fg_pixels: '?' 
    gether_inference_stats: '?'
    summary_compute_global_metrics: '?' 
    log_interval: 20 
    show_examples: False 
    log_image_stats: True 
    log_pixel_stats: True 
    track_ensemble_member_scores: True

experiment:
    exp_root: '?'
    seed: 42

data:
    splits: ('val', 'cal')
    preload: False 

dataloader:
    batch_size: 1 
    num_workers: 0
    pin_memory: True 

model:
    calibrator: '?'
    checkpoint: '?' 
    pretrained_exp_root : '?' 
    pretrained_select_metric: "val-dice_score"

<IPython.core.display.Javascript object>

In [3]:
%%yaml ensemble_cfg

ensemble:
    combine_fn: '?'
    combine_quantity: '?'
    member_w_metric: 'None'

<IPython.core.display.Javascript object>

In [4]:
additional_args = {
    "log.gether_inference_stats": [True],
    "log.summary_compute_global_metrics": [True]
}

In [5]:
exp_name = '02_19_24_WMH_SmoothNectar' 
group_dict = {
    "dataset": "WMH",
    "exp_group": exp_name,
    "base_models_group": "01_08_24_WMH_Ensemble",
    "calibrated_models_group": "01_19_24_WMH_Foreground_Calibrators",
    "preload": False
}

# exp_name = '02_19_24_CityScapes_SmoothNectar' 
# group_dict = {
#     "dataset": "CityScapes",
#     "exp_group": exp_name,
#     "base_models_group": "01_25_24_CityScapes_Dice",
#     "calibrated_models_group": "01_26_24_CityScapes_Ensemble",
#     "preload": False
# }

### Standard Calibrators.

In [6]:
do_ensemble = False 
inference_func = get_cal_stats
# Get the configs for the different runs.
option_set = get_ese_inference_configs(
    group_dict=group_dict,
    calibrators_list=[
        # 'Uncalibrated',
        # 'Vanilla',
        # 'TempScaling',
        # 'VectorScaling',
        # 'DirichletScaling',
        # 'LTS',
        'NectarScaling',
    ], 
    do_ensemble=do_ensemble, 
    additional_args=additional_args
)

### Binning Calibrators

In [7]:
# do_ensemble = True 
# inference_func = get_cal_stats
# # Get the configs for the different runs.
# option_set = get_ese_inference_configs(
#     group_dict=group_dict,
#     calibrators_list=[
#         # 'HistogramBinning',
#         'NectarBinning',
#         'SoftNectarBinning'
#     ], 
#     do_ensemble=do_ensemble, 
#     normalize_opts=[
#         False
#     ],
#     cal_stats_splits=[
#         'val',
#         'cal'
#     ],
#     additional_args=additional_args
# )

### Upper Bounds

In [8]:
# # # Get the configs for computing the upepr bounds
# inference_func = get_ensemble_ub
# option_set = get_ese_inference_configs(
#     group_dict=group_dict,
#     do_ensemble=True, 
#     ensemble_upper_bound=True,
#     calibrators_list=[
#         'Uncalibrated',
#     ], 
#     ens_cfg_options=[
#         ('upperbound', 'probs')
#     ]
# )

In [9]:
# Load the inference cfg from local.
##################################################
inf_cfg_root = code_root / "ese" / "experiment" / "configs" / "inference"
##################################################
with open(inf_cfg_root / f"{group_dict['dataset']}.yaml", 'r') as file:
    dataset_inference_cfg = yaml.safe_load(file)
with open(inf_cfg_root / "Calibration_Metrics.yaml", 'r') as file:
    cal_metrics_cfg = yaml.safe_load(file)
##################################################
base_cfg = Config(inference_config).update([dataset_inference_cfg, cal_metrics_cfg])
if do_ensemble:
    base_cfg = base_cfg.update(ensemble_cfg)

# Get the configs
cfgs = get_option_product(exp_name, option_set, base_cfg)

In [10]:
len(cfgs)

4

## Running Jobs

In [11]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [12]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=cfgs[0], 
    job_func=get_cal_stats,
    run_name='debug',
    show_examples=False,
    gpu='3',
) 

Set seed: 42


/storage/vbutoi/projects/ionpy/util/libcheck.py:49: UserWarning: Intel MKL extensions not available for NumPy
  warn("Intel MKL extensions not available for NumPy")
/storage/vbutoi/projects/ionpy/util/libcheck.py:57: UserWarning: Using slow Pillow instead of Pillow-SIMD
  warn("Using slow Pillow instead of Pillow-SIMD")


Set seed: 42
Set seed: 42
Running:

data:
  preload: false
  slicing: dense_full
  splits: ('val', 'cal')
dataloader:
  batch_size: 1
  num_workers: 0
  pin_memory: true
dataset:
  _class: ese.experiment.datasets.WMH
  annotator: observer_o12
  augmentations: null
  axis: 0
  num_slices: 1
  preload: false
  return_data_id: true
  slicing: dense_full
  task: Amsterdam
  version: 0.2
experiment:
  dataset_name: WMH
  exp_root: /storage/vbutoi/scratch/ESE/inference/02_19_24_WMH_SmoothNectar
  pretrained_seed: 42
  seed: 42
global_cal_metrics:
- ECE:
    _fn: ese.experiment.metrics.ece.ece_loss
    cal_type: toplabel
- Edge-ECE:
    _fn: ese.experiment.metrics.ece.edge_ece_loss
    cal_type: toplabel
- CW-ECE:
    _fn: ese.experiment.metrics.ece.cw_ece_loss
    cal_type: classwise
    class_weighting: proportional
- ECW-ECE:
    _fn: ese.experiment.metrics.ece.ecw_ece_loss
    cal_type: classwise
    class_weighting: proportional
- Uniform-CW-ECE:
    _fn: ese.experiment.metrics.ece.cw_ec

TypeError: unexpected keyword arguments: 'start', 'end'

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 
# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
# )